In [ ]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle

In [ ]:
local_crs = 3414

In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
# with open('./out/singapore/what_cells_are_in_what_cluster_dict.json', 'r') as f:
#     includes_dict = json.load(f)
    
tessellation = gpd.read_parquet("./out/singapore/tessellation.pq")

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

In [ ]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
### Distance Weight
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

In [ ]:
# @dask.delayed
# def queen_contiguity_load(n1, includes_dict):
#     chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{int(n1)}.pq")
    
#     # weights
    # queen_1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    # queen_3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
#     return (queen_1, queen_3)

In [ ]:
# file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
# file_list = glob.glob(file_pattern)
# num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

# weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

# weight_out = dask.compute(weight_out)


In [ ]:
# queen_1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

# queen_3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [ ]:
tessellation_bridged = gpd.read_parquet("./out/singapore/tessellation_bridges.pq")

In [ ]:
queen_1 = libpysal.weights.fuzzy_contiguity(tessellation_bridged, tolerance=0.05, buffering=True, drop=True, buffer=20, ids="uID", silence_warnings=True)
queen_3 = momepy.sw_high(k=3, weights=queen_1)

In [ ]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [ ]:
### Building Area
buildings['area'] = momepy.Area(buildings).series

## Building Shapes

### Circular Compactness
buildings['circular_com'] =  momepy.CircularCompactness(buildings).series

### Elongation
blg_elongation = momepy.Elongation(buildings)
buildings['elongation'] = blg_elongation.series

## Squareness
blg_squareness = momepy.Squareness(buildings)
buildings['squareness'] = blg_squareness.series

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

buildings['area'] = momepy.Area(buildings).series


buildings['orientation'] = momepy.Orientation(buildings).series

In [ ]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

In [ ]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [ ]:
%%time
# momepy.node_degree(primal)

In [ ]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

In [ ]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

In [ ]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [ ]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [ ]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [ ]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [ ]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [ ]:
# Find the uIDs in tessellation that are not in buildings
tessellation_uids = set(tessellation['uID'])
buildings_uids = set(buildings['uID'])
new_uids = buildings_uids - tessellation_uids

# Create a new GeoDataFrame with the new uIDs and empty geometry
new_rows = pd.DataFrame({'uID': list(new_uids)})
new_rows['geometry'] = None
new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# Append the new GeoDataFrame to buildings
tessellation = pd.concat([tessellation, new_gdf], ignore_index=True)

In [ ]:
tessellation["convexity"] = momepy.Convexity(tessellation).series
tessellation["area"] = momepy.Area(tessellation).series

tessellation["neighbours"] = momepy.Neighbors(tessellation, queen_3,'uID').series
tessellation["neighbours400"] = momepy.Neighbors(tessellation, dist200, 'uID').series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID", verbose=False).series

areas = momepy.Area(tessellation).series
tessellation["mean_area"] = momepy.AverageCharacter(
tessellation, values=areas, spatial_weights=queen_3, unique_id='uID').mean

tessellation["mean_area_iq"] = momepy.AverageCharacter(
    tessellation, areas, queen_3, 'uID', rng=(25, 75)).mean

tessellation["area_rng"] = momepy.Range(tessellation, values='area',
                                        spatial_weights=queen_3, unique_id='uID').series
tessellation["area_iqr"] = momepy.Range(tessellation, values='area',
                        spatial_weights=queen_3, unique_id='uID',
                        rng=(25, 75)).series
tessellation["rea_theil"] = momepy.Theil(tessellation, values='area',
                        spatial_weights=queen_3,
                        unique_id='uID').series

tessellation["shannon"] = momepy.Shannon(tessellation, values='area',spatial_weights=queen_3, unique_id='uID').series


In [ ]:
circular_compactness = momepy.CircularCompactness(buildings)
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

buildings['neighbour_dist'] = momepy.NeighborDistance(buildings, queen_1, 'uID').series

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, queen_1, 'uID', 3, verbose=False).series

buildings['mean_ib_dist'] = momepy.MeanInterbuildingDistance(
    buildings, queen_1, 'uID', order=3).series




In [ ]:
cols = list(streets.columns)
cols.remove('geometry')

In [ ]:
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, queen_3, 'uID', momepy.Area(buildings).series).series

In [ ]:
streets_dropped = streets.drop(cols,axis=1).reset_index(drop=True)

In [ ]:
buildings = buildings.set_crs(local_crs)

In [ ]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=streets_dropped, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [ ]:
buildings['orientation'] = momepy.Orientation(buildings).series
tessellation['orientation'] = momepy.Orientation(tessellation).series

In [ ]:
streets.reset_index(drop=True, inplace=True)

In [ ]:
streets["nID"]= streets.index

In [ ]:
buildings.drop(columns=['index_right'], inplace=True)

In [ ]:
buildings =  gpd.sjoin_nearest(buildings, streets, how="left", distance_col=1000)

In [ ]:
str_align = momepy.StreetAlignment(buildings, streets,
                                   'orientation', 'nID',
                                   'nID').series

In [ ]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'orientation', 'orientation',
                                      'uID', 'uID')
buildings['cell_align'] = blg_cell_align.series

In [ ]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

In [ ]:
nodes = gpd.read_parquet("./out/singapore/vertices_frame.pq")
edges = gpd.read_parquet("./out/singapore/edges_frame.pq")

In [ ]:
buildings = buildings.drop(["nID", "index_right", "nID"], axis=1)
nodes = nodes.drop(["vertID"], axis=1)
edges = edges.drop("edgeID", axis=1)

In [ ]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=nodes, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
buildings = buildings.drop("index_right", axis=1)
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=edges, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


In [ ]:
tessellation = tessellation.merge(buildings, how='left', left_on = "uID", right_on="uID" )
tessellation = tessellation.drop(['eID', "geometry_y", "index_right"], axis=1)
tessellation = tessellation.rename(columns={"geometry_x": "geometry"})
tessellation = gpd.GeoDataFrame(tessellation.dropna(subset=['geometry']))

In [ ]:
with open('./out/singapore/weights.pickle', 'wb') as f:
    pickle.dump(queen_1, f)
    pickle.dump(queen_3, f)
    pickle.dump(dist200, f)
    f.close()

In [ ]:
tessellation.columns=tessellation.columns.astype(str)

In [ ]:
tessellation.to_parquet("./out/singapore/tessellation_stats.pq")

In [ ]:
# chunk_size = 250000
# chunks = []
# for i in range(0, len(tessellation), chunk_size):
#     chunks.append(tessellation.iloc[i:i + chunk_size])

In [ ]:
# import os
# new_directory = "./out/singapore/tessellation/"

# if not os.path.exists(new_directory):
#     os.makedirs(new_directory)
#     print(f"Created directory: {new_directory}")
# else:
#     print(f"Directory already exists: {new_directory}")

# # iterate over all files in the directory and delete them
# for filename in os.listdir(new_directory):
#     file_path = os.path.join(new_directory, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted file: {file_path}")

In [ ]:
# for i, chunk in enumerate(chunks):
#     chunk.to_parquet(f"./out/singapore/tessellation/tesselation_chunk_{i}.pq")

In [ ]:
# duplicates = tessellation[tessellation.duplicated(['uID'], keep=False)]
# if len(duplicates) > 0:
#     print(f"There are {len(duplicates)} rows with duplicate uids.")
#     print(duplicates)
# else:
#     print("There are no rows with duplicate uids.")

In [ ]:
# import pickle

# # Open the file for reading
# with open('./out/singapore/weights.pickle', 'rb') as f:
#     # Load the variables in the same order that they were dumped
#     queen_1 = pickle.load(f)
#     queen_3 = pickle.load(f)
#     dist200 = pickle.load(f)
#     tessellation = pickle.load(f)

In [ ]:
# tessellation.to_parquet("./out/singapore/tessellation_with_data.pq", write_metadata_file=False)